In [32]:
import sys
import os

#Add the path from where src is available OR change it to `sys.path.append(os.path.realpath("."))`
if '../../../' not in sys.path:
    sys.path.insert(0,'../../../')
    
from src.algorithms.sentiment import get_sentiment

In [70]:
corpus = ['I have for a while now really only used Mint for an aggregator of my transactions and balances due to not using exact numbers but now they appear to have taken that away from the account balances too. Mint is good for people who are bad at budgeting but there are a lot better ways to budget',
           'Why does the new app round up the account balance rather than show the exact amount now? The balance on every account is to the next dollar higher rather than to the exact penny the version before.  Accuracy is important.   This is a bad feature on the new update',
           'The recent upgrade doesn’t show actual amounts in accounts. It rounds to the nearest dollar.',
           'The app does what its supposed to do. Need an uplift in the UI. Basically the colors look really dull. Also need options to create and manage new categories.',
           'Two hours of trying to delete this account and it is not working. I followed the online instructions but the link did not work, tried from inside the account, talked to chat support but she couldn’t figure it out, now I am on the phone AGAIN hoping for a miracle which is unlikely since I am told there are many people having problems with the app',
         'I have only had the app for 2 days, and it is already not working. I am trying to add a transaction, but it says “Unable to connect to server.” Everything else is working fine on my phone.',
         'Used to work great. Now it says “uhoh can’t connect to the server” even though it obviously can since it loads new transactions. Much much slower than before. Can’t see accounts because pressing anywhere in that area automatically loads their new net worth chart']

In [75]:
import nltk
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
tokenized_corpus=[]
num=0
for i in corpus:
    temp_list= tokenizer.tokenize(i) #use better way
    for j in temp_list:
        tokenized_corpus.append(j)
        print(str(num)+" "+j)
        num=num+1

# print(tokenized_corpus)
# print(len(tokenized_corpus))


#We lose the importance of few sentences here. can i use text summarizer here to reduce the length of sentences ? 

0 I have for a while now really only used Mint for an aggregator of my transactions and balances due to not using exact numbers but now they appear to have taken that away from the account balances too.
1 Mint is good for people who are bad at budgeting but there are a lot better ways to budget
2 Why does the new app round up the account balance rather than show the exact amount now?
3 The balance on every account is to the next dollar higher rather than to the exact penny the version before.
4 Accuracy is important.
5 This is a bad feature on the new update
6 The recent upgrade doesn’t show actual amounts in accounts.
7 It rounds to the nearest dollar.
8 The app does what its supposed to do.
9 Need an uplift in the UI.
10 Basically the colors look really dull.
11 Also need options to create and manage new categories.
12 Two hours of trying to delete this account and it is not working.
13 I followed the online instructions but the link did not work, tried from inside the account, talke

In [36]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/rjain13/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [37]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rjain13/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [42]:
# wget http://nlp.stanford.edu/data/glove.6B.zip
import numpy as np
import pandas as pd

In [46]:
# tokenized_corpus[:5]

clean_sentences = [s.lower() for s in tokenized_corpus]

from nltk.corpus import stopwords
stop_words = stopwords.words('english')


def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
clean_sentences[:5]


['really used mint aggregator transactions balances due using exact numbers appear taken away account balances too.',
 'mint good people bad budgeting lot better ways budget',
 'new app round account balance rather show exact amount now?',
 'balance every account next dollar higher rather exact penny version before.',
 'accuracy important.']

In [49]:
word_embeddings = {}
f = open('./glove.6B/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [50]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)


In [51]:
sentence_vectors[0].shape


(100,)

In [53]:
#similarity matrix
sim_mat = np.zeros([len(tokenized_corpus), len(tokenized_corpus)])


In [54]:
from sklearn.metrics.pairwise import cosine_similarity


In [55]:
for i in range(len(tokenized_corpus)):
    for j in range(len(tokenized_corpus)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]


In [78]:
sim_mat[15][17]
sim_mat

array([[0.        , 0.81012792, 0.91362006, 0.88630339, 0.48519017,
        0.76244778, 0.8704305 , 0.43990685, 0.59698878, 0.5642635 ,
        0.72084478, 0.83275762, 0.82859927, 0.83512511, 0.6827445 ,
        0.78746461, 0.82302333, 0.84912189, 0.7383407 , 0.89015017],
       [0.81012792, 0.        , 0.83306258, 0.83325679, 0.36359519,
        0.79590702, 0.78337614, 0.38127525, 0.55269596, 0.69447734,
        0.82236877, 0.84262656, 0.74974163, 0.82015941, 0.61985838,
        0.87639089, 0.74758338, 0.86192588, 0.79602204, 0.80620878],
       [0.91362006, 0.83306258, 0.        , 0.93074381, 0.47120345,
        0.83892736, 0.9148176 , 0.48616387, 0.67083821, 0.61517096,
        0.72704985, 0.868679  , 0.83235071, 0.87028904, 0.7614577 ,
        0.81306136, 0.7882218 , 0.87345995, 0.76841266, 0.89918647],
       [0.88630339, 0.83325679, 0.93074381, 0.        , 0.42428283,
        0.78072963, 0.8748894 , 0.45856297, 0.55637585, 0.59035111,
        0.7094066 , 0.81513607, 0.79041385, 0

In [57]:
import matplotlib.pyplot as plt
%matplotlib inline


In [58]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)


In [61]:
print(nx_graph)

In [62]:
print(len(scores))


20


In [63]:
scores


{0: 0.05510769650888121,
 1: 0.05396530006183286,
 2: 0.056991095914136744,
 3: 0.05478274994039148,
 4: 0.02983634510374607,
 5: 0.05182676768387081,
 6: 0.054240183234886455,
 7: 0.03242414199079087,
 8: 0.042249281777298,
 9: 0.0422336473795981,
 10: 0.04848536670475073,
 11: 0.0545921979364666,
 12: 0.052823823779423316,
 13: 0.05547276848823688,
 14: 0.04729844196509034,
 15: 0.053937894375810765,
 16: 0.051388041149270355,
 17: 0.056495233768338426,
 18: 0.05040835585802234,
 19: 0.05544066637915757}

In [64]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(tokenized_corpus)), reverse=True)


In [65]:
type(ranked_sentences)


list

In [66]:
y = len(ranked_sentences)
print(y)


20


In [68]:
# Extract top 5 sentences as the summary
for i in range(3):
    print(ranked_sentences[i][1])
    print()

Why does the new app round up the account balance rather than show the exact amount now?

Now it says “uhoh can’t connect to the server” even though it obviously can since it loads new transactions.

I followed the online instructions but the link did not work, tried from inside the account, talked to chat support but she couldn’t figure it out, now I am on the phone AGAIN hoping for a miracle which is unlikely since I am told there are many people having problems with the app



In [82]:
from summa import keywords
from summa.summarizer import summarize
text="Used to work great. Now it says “uhoh can’t connect to the server” even though it obviously can since it loads new transactions. Much much slower than before. Can’t see accounts because pressing anywhere in that area automatically loads their new net worth chart. I have only had the app for 2 days, and it is already not working. I am trying to add a transaction, but it says “Unable to connect to server.” Everything else is working fine on my phone."


#apply logic to have max 2-3 lines, if greater than that, count words and do/2 with max limit 50
print(len(tokenizer.tokenize(i)))
print((2/len(tokenizer.tokenize(i))))
print(len(text.split()))
a=summarize(text,words=40)
print(a)

print(keywords.keywords(text))


4
0.5
82
Now it says “uhoh can’t connect to the server” even though it obviously can since it loads new transactions.
I am trying to add a transaction, but it says “Unable to connect to server.” Everything else is working fine on my phone.
work
working
great
